In [1]:
import pandas as pd
import numpy as np


In [2]:
def categorizar_valores_faltantes(categorias):
    categorias = categorias.copy()

    # Reemplazar valores faltantes en EstadoCobro
    categorias["EstadoCobro"] = categorias["EstadoCobro"].replace([np.nan, "N/A", "NULL"], "Sin gestión")

    # Reemplazar valores faltantes en Factura
    categorias["Factura"] = categorias["Factura"].replace([np.nan, "N/A", "NULL"], "Sin factura")

    return categorias



In [3]:
cobranza_final = pd.read_excel("data/tabla_ejemplo_clientes.xlsx")

cobranza_final = categorizar_valores_faltantes(cobranza_final)


In [5]:
print(cobranza_final["EstadoCobro"].value_counts(dropna=False))
print("Fechas faltantes:", cobranza_final["FechaFactura"].isna().sum())

EstadoCobro
Sin gestión             608
Pagado                  261
Facturado                51
Validar beneficio        14
Solicitar aprobación      3
Cobrar luego              1
Name: count, dtype: int64
Fechas faltantes: 626


In [10]:
# Se calculan los días desde la fecha de factura
hoy = pd.to_datetime("today")
cobranza_final["DiasDesdeFactura"] = (hoy - cobranza_final["FechaFactura"]).dt.days

condiciones = [
    (cobranza_final["EstadoCobro"] == "Pagado"), 
    (cobranza_final["EstadoCobro"] == "Solicitar aprobación") & (cobranza_final["FechaFactura"].isna()),
    (cobranza_final["ValorFacturado"].notna()) & 
    (cobranza_final["ValorFacturado"] > 0) & 
    (cobranza_final["FechaFactura"].isna()) & 
    (~cobranza_final["EstadoCobro"].isin(["Pagado", "Facturado"])),
    (cobranza_final["FechaFactura"].notna()) & 
    (cobranza_final["DiasDesdeFactura"] > 20) & 
    (cobranza_final["EstadoCobro"] != "Pagado"),
    (cobranza_final["FechaFactura"].notna()) & 
    (cobranza_final["DiasDesdeFactura"].between(15, 20)) & 
    (cobranza_final["EstadoCobro"] != "Pagado")
]

resultados = [
    "Pagado",  
    "Solicitar aprobación",
    "Sin fecha ni estado",
    "Factura vencida",
    "Próxima a vencer"
]

cobranza_final["TipoGestion"] = np.select(condiciones, resultados, default="Sin gestión")


In [11]:
# Conteo por tipo de gestión
print(cobranza_final["TipoGestion"].value_counts())

TipoGestion
Sin gestión             619
Pagado                  261
Factura vencida          43
Sin fecha ni estado       8
Próxima a vencer          4
Solicitar aprobación      3
Name: count, dtype: int64


In [12]:
comprobar = cobranza_final.groupby("TipoGestion").head(2)
display(comprobar)

,ID,IDRef,ValorFacturado,ValorRecuperado,EstadoCobro,Factura,FechaFactura,DiasDesdeFactura,TipoGestion
0,720,1,4.036085e+05,1.009023e+06,Facturado,FVE168,2025-06-17,10.0,Sin gestión
1,715,1,5.063535e+05,1.265883e+06,Facturado,FVE166,2025-06-13,14.0,Sin gestión
4,704,1,8.123602e+05,2.030901e+06,Facturado,FVE162,2025-06-09,18.0,Próxima a vencer
5,710,1,1.532956e+06,6.144995e+06,Facturado,FVE162,2025-06-09,18.0,Próxima a vencer
8,706,1,4.753056e+05,2.127837e+06,Facturado,FVE159,2025-06-05,22.0,Factura vencida
9,707,1,1.083734e+05,2.709328e+05,Facturado,FVE159,2025-06-05,22.0,Factura vencida
20,709,1,5.703099e+05,1.425776e+06,Pagado,FVE154,2025-05-19,39.0,Pagado
24,579,1,1.244967e+05,1.867449e+06,Pagado,FVE155,2025-05-19,39.0,Pagado
399,236,1,8.012294e+03,4.006024e+04,Sin gestión,Sin factura,NaT,NaN,Sin fecha ni estado
400,236,2,6.488847e+03,3.244547e+04,Sin gestión,Sin factura,NaT,NaN,Sin fecha ni estado


In [14]:
cobranza_final.to_excel("cobranza_final.xlsx", index=False)
